In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/data_raw.csv')

/Users/hugorebeix/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (95,96,97,98,100,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,120,122) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data.shape

(870492, 124)

In [4]:
describe = data.describe(include='all')

In [5]:
describe

,Unnamed: 0,Country Name,Country Code,Indicator Name_x,Indicator Code,1970,1971,1972,1973,1974,...,Latest household survey,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data,Unnamed: 31,DESCRIPTION,Unnamed: 3
count,870492.000000,870492,870492,870492,870492,7.148000e+04,3.444300e+04,3.451400e+04,3.446700e+04,3.467500e+04,...,509292,577920,400932,512904.0,386484.000000,668220.000000,646548.0,0.0,613,0.0
unique,NaN,241,241,3612,3612,NaN,NaN,NaN,NaN,NaN,...,60,75,2,44.0,NaN,NaN,21.0,NaN,97,NaN
top,NaN,Italy,MYS,"Barro-Lee: Population in thousands, age 30-34,...",LO.LLECE.REA6.P50,NaN,NaN,NaN,NaN,NaN,...,"Demographic and Health Survey (DHS), 2013","Integrated household survey (IHS), 2012",Yes,2010.0,NaN,NaN,2005.0,NaN,Data sources : United Nations World Population...,NaN
freq,NaN,3612,3612,241,241,NaN,NaN,NaN,NaN,NaN,...,36120,54180,397320,82920.0,NaN,NaN,144480.0,NaN,154,NaN
mean,435245.500000,NaN,NaN,NaN,NaN,1.997095e+09,4.388745e+09,4.739394e+09,5.264671e+09,5.565835e+09,...,NaN,NaN,NaN,NaN,2008.102804,2010.994595,NaN,NaN,NaN,NaN
std,251289.539602,NaN,NaN,NaN,NaN,1.218514e+11,1.833238e+11,1.944465e+11,2.091105e+11,2.144020e+11,...,NaN,NaN,NaN,NaN,2.604581,2.562723,NaN,NaN,NaN,NaN
min,0.000000,NaN,NaN,NaN,NaN,-1.435564e+00,-1.594625e+00,-3.056522e+00,-4.032582e+00,-4.213563e+00,...,NaN,NaN,NaN,NaN,2000.000000,1995.000000,NaN,NaN,NaN,NaN
25%,217622.750000,NaN,NaN,NaN,NaN,8.700000e-01,1.045615e+01,1.076718e+01,1.116327e+01,1.131197e+01,...,NaN,NaN,NaN,NaN,2007.000000,2011.000000,NaN,NaN,NaN,NaN
50%,435245.500000,NaN,NaN,NaN,NaN,6.570000e+00,7.207677e+01,7.685992e+01,7.963683e+01,8.090797e+01,...,NaN,NaN,NaN,NaN,2009.000000,2012.000000,NaN,NaN,NaN,NaN
75%,652868.250000,NaN,NaN,NaN,NaN,6.387674e+01,6.452600e+04,6.653125e+04,6.973650e+04,7.127600e+04,...,NaN,NaN,NaN,NaN,2010.000000,2012.000000,NaN,NaN,NaN,NaN


Retirons les colonnes vides

In [6]:
counts = describe.iloc[0]

In [7]:
counts

Unnamed: 0                      870492
Country Name                    870492
Country Code                    870492
Indicator Name_x                870492
Indicator Code                  870492
                                 ...  
Latest trade data               668220
Latest water withdrawal data    646548
Unnamed: 31                          0
DESCRIPTION                        613
Unnamed: 3                           0
Name: count, Length: 124, dtype: object

In [8]:
useless_cols = counts[counts == 0]

In [9]:
useless_cols

Unnamed: 69                   0
Unit of measure               0
Notes from original source    0
Other web links               0
Related indicators            0
License Type                  0
Unnamed: 20                   0
Unnamed: 31                   0
Unnamed: 3                    0
Name: count, dtype: object

In [10]:
data = data.drop(columns=useless_cols.index)

In [11]:
data.shape

(870492, 115)

### Parmis les 3600 indicateurs, il faudrait selectionner ceux qui sont en relation avec notre problématique "métier".
#### On va donc établir une liste de mots clés en relation avec cette problématique.
#### On recherche des pays dont le marché est propice à un developpement de l'activité d'une plateforme d'apprentissage en ligne pour les 15 - 30 ans (en prenant large). On va donc chercher des mots clés liés à l'age, à l'accès internet, à l'éducation.

In [12]:
cols = data.columns
cols

Index(['Unnamed: 0', 'Country Name', 'Country Code', 'Indicator Name_x',
       'Indicator Code', '1970', '1971', '1972', '1973', '1974',
       ...
       'IMF data dissemination standard', 'Latest population census',
       'Latest household survey',
       'Source of most recent Income and expenditure data',
       'Vital registration complete', 'Latest agricultural census',
       'Latest industrial data', 'Latest trade data',
       'Latest water withdrawal data', 'DESCRIPTION'],
      dtype='object', length=115)

In [13]:
indicateurs = data['Indicator Name_x'].unique()

In [14]:
indicateurs[:5]

array(['Adjusted net enrolment rate, lower secondary, both sexes (%)',
       'Adjusted net enrolment rate, lower secondary, female (%)',
       'Adjusted net enrolment rate, lower secondary, gender parity index (GPI)',
       'Adjusted net enrolment rate, lower secondary, male (%)',
       'Adjusted net enrolment rate, primary, both sexes (%)'],
      dtype=object)

In [15]:
key_words = ['internet', 'young', 'education',
             'access', 'age', 'diploma',
             '15', 'old', '30', 
             'enrolment', 'secondary', 'primary',
             'illiteracy', 'growth', 'professor']

In [16]:
indicateurs_interessants = []

for indicateur in indicateurs:
    mots = indicateur.split(' ')
    for mot in mots:
        if mot in key_words:
            indicateurs_interessants.append(indicateur)
            break # pour éviter les doublons
len(indicateurs_interessants)

1639

La solution ci dessus m'a permis de controler la solution ci dessous. Cette fonction iterrows me donnait que len(lignes_pas_interessantes)=475493

In [17]:
%%timeit
def test_mapper(name):
    if name in indicateurs_interessants:
        return True
    else:
        return False

data['interessant'] = data['Indicator Name_x'].map(test_mapper)

16.2 s ± 536 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
data_test = data[data['interessant']]

In [19]:
data_test.shape

(394999, 116)

In [20]:
data.shape

(870492, 116)

Je vérifie bien que le filtre a fonctionné

In [21]:
data_test.shape[0] + 475493 == data.shape[0]

True

In [22]:
data = data_test

In [24]:
data = data.drop(columns=['interessant'])

In [25]:
data.shape

(394999, 115)

In [27]:
print(data.columns)

Index(['Unnamed: 0', 'Country Name', 'Country Code', 'Indicator Name_x',
       'Indicator Code', '1970', '1971', '1972', '1973', '1974',
       ...
       'IMF data dissemination standard', 'Latest population census',
       'Latest household survey',
       'Source of most recent Income and expenditure data',
       'Vital registration complete', 'Latest agricultural census',
       'Latest industrial data', 'Latest trade data',
       'Latest water withdrawal data', 'DESCRIPTION'],
      dtype='object', length=115)


Il reste encore beaucoup trop de colonnes.